In [20]:
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
import importlib
importlib.reload(zeta_py)

<module 'zeta_py' from '/Users/tfrizza/Documents/zeta/zeta-py/zeta_py/__init__.py'>

In [1]:
from zeta_py.zeta_client.accounts.pricing import Pricing
from solana.rpc.async_api import AsyncClient
from zeta_py.client import Client
from zeta_py.types import Network, Asset
import anchorpy
from solders.pubkey import Pubkey
from zeta_py import pda

conn = AsyncClient("https://api.mainnet-beta.solana.com")

wallet = anchorpy.Wallet.local() # get local filesystem keypair wallet
print(wallet.public_key)

4mYxEXiFwe3ePEmoj7CYCbiW7wGTojUQCmdPqGaoVaUh


In [2]:
print(f"Balance: {(await conn.get_balance(wallet.public_key)).value / 1e9} SOL")

In [3]:
client = await Client.create(Network.MAINNET, conn, wallet, [Asset.SOL])

Loaded account: State
Loaded account: Pricing
Loaded account: PerpSyncQueue
Loaded account: Clock
Loaded account: CrossMarginAccount


In [8]:
print(client.balance)
print(client.positions)
print(client.open_orders)

0.009936
{<Asset.SOL: 0>: Position(size=0.001, cost_of_trades=0.021296)}
{<Asset.SOL: 0>: [Order(order_id=391598569689894661041978810, client_id=0, open_order_address=Pubkey(
    ciwBQfLdM97vFp8GwYYLczPgYoLfdkTVD6dhHAeGoWj,
), open_order_slot=0, fee_tier=0, info=OrderInfo(price=21.2286, size=0.001, price_lots=21228600, size_lots=1), side=<Side.BID: 0>, tif_offset=0)]}
